# Create sampling points for Rhino and Modelica

Stupid random sampling. More systematic ways: explore pyDOE! 
- https://pythonhosted.org/pyDOE/index.html
- https://github.com/clicumu/pyDOE2

How to identify things

- Project (projectid: Kiss1)
- Study: a variant within the parameters of the project (studyid -> vergeben vom Optimierer oder wie hier)
- Geometry Model (parametric) -> github commit id nutzen
- Geometry Instance -> id von rhino vergeben
- Analysis model (parametric) -> github commit id nutzen
- analysis instance -> von analyse vergeben

geometrie ist auch eine analyse!!

Welche Rolle spielt CAD? nur am ende zur Doku...

### Input parameters (for Rhino)

- number of racks
- number of cells per rack
- cooling channel width
- cooling channel height
- wall thickness

### Input parameters for OpenModelica

- cover (wall between cell and pipe) 
  - area_h = 3.44e-4, 
  - c_wall = 888, 
  - k_wall = 220,
  - rho_wall = 2700, 
  - s = 0.002
- "wall" between two cells
  - area_h = 1.72e-4, 
  - c_wall = 888, 
  - k_wall = 220, 
  - rho_wall = 2700, 
  - s = 0.04
- pipe
  - crossArea = 0.32e-4, 
  - length = 0.043, 
  - perimeter = 0.024, 
- mass flow source
  - m_flow = 0.0037,
- Gc_Thermischer_Kontaktwiderstand(k = 3.311) 



In [60]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from uuid import uuid1
import json
import random


In [61]:
# constants
constants = {
    "C_quartercell": 36.3,
    "k_contact": 3.311,  # TODO: rename to k_cell

    "c_cover": 888.,
    "k_cover": 220.,
    "rho_cover": 2700.,

    "c_plate": 888.,
    "k_plate": 220.,
    "rho_plate": 2700.,

    "channel_dist": 0.04,

    "cell_width": 0.043,
    "stoptime": 2000,
}

# Configurations for number of racks and number of cells. 
# total number of cells in a battery pack is 2*nracks*ncells
# varies among 140 and 144
# key: nracks, value: ncells
cellconfigs = {2: 35, 3: 24, 4: 18, 5: 14, 6: 12, 7: 10, 8: 9,
               9: 8, 10: 7, 12: 6, 14: 5, 18: 4, 24: 3, 35: 2}

In [69]:
# number of points to sample
N = 200

df = pd.DataFrame(data={
    "t_cover": [random.uniform(0.5e-3, 2.0e-3) for _ in range(N)],
    "channel_width": [random.uniform(4e-3, 19e-3) for _ in range(N)], 
    "channel_height": [random.uniform(0.5e-3, 6e-3) for _ in range(N)],
    "nracks": random.choices(list(cellconfigs.keys()), k=N)
})

# compute ncells
df["ncells"] = df.nracks.apply(lambda nr: cellconfigs[nr])

# add constants
for k, v in constants.items():
    df[k] = v
    
# constant m_flow !!!
df['m_flow'] = 120/3600/df['nracks']
    
# drop rows with w < h
df.query("channel_width >= channel_height", inplace=True)
    
# compute modelica stuff
df["area_cover"] = df["channel_width"] * df["cell_width"]
df["t_plate"] = df["channel_dist"] - df["channel_width"]
df["area_plate"] = df["channel_height"] * df["cell_width"] #  + 2 * t_cover??
df["area_channel"] = df["channel_height"] * df["channel_width"]
df["perim_channel"] = 2 * (df["channel_height"] + df["channel_width"])

# create simulation id
df["simid"] = [str(uuid1()) for _ in range(len(df))]

In [70]:
df.describe().T

count         mean           std          min          25%  \
t_cover         198.0     0.001265  4.091223e-04     0.000519     0.000874   
channel_width   198.0     0.011770  4.455633e-03     0.004016     0.007603   
channel_height  198.0     0.003074  1.499768e-03     0.000507     0.001791   
nracks          198.0    11.176768  8.568865e+00     2.000000     6.000000   
ncells          198.0    10.828283  8.352076e+00     2.000000     5.000000   
C_quartercell   198.0    36.300000  1.424688e-14    36.300000    36.300000   
k_contact       198.0     3.311000  4.452149e-16     3.311000     3.311000   
c_cover         198.0   888.000000  0.000000e+00   888.000000   888.000000   
k_cover         198.0   220.000000  0.000000e+00   220.000000   220.000000   
rho_cover       198.0  2700.000000  0.000000e+00  2700.000000  2700.000000   
c_plate         198.0   888.000000  0.000000e+00   888.000000   888.000000   
k_plate         198.0   220.000000  0.000000e+00   220.000000   220.000000   
rho_plate       198.0  2700.000000  0.000000e+00  2700.000000  2700.000000   
channel_dist    198.0     0.040000  6.956483e-18     0.040000     0.040000   
cell_width      198.0     0.043000  0.000000e+00     0.043000     0.043000   
stoptime        198.0  2000.000000  0.000000e+00  2000.000000  2000.000000   
m_flow          198.0     0.005074  3.943501e-03     0.000952     0.002381   
area_cover      198.0     0.000506  1.915922e-04     0.000173     0.000327   
t_plate         198.0     0.028230  4.455633e-03     0.020026     0.024402   
area_plate      198.0     0.000132  6.449002e-05     0.000022     0.000077   
area_channel    198.0     0.000036  2.313273e-05     0.000004     0.000017   
perim_channel   198.0     0.029688  9.309085e-03     0.011390     0.021870   

                        50%          75%          max  
t_cover            0.001262     0.001595     0.001996  
channel_width      0.011918     0.015598     0.019974  
channel_height     0.003098     0.004205     0.005950  
nracks             9.000000    14.000000    35.000000  
ncells             8.000000    12.000000    35.000000  
C_quartercell     36.300000    36.300000    36.300000  
k_contact          3.311000     3.311000     3.311000  
c_cover          888.000000   888.000000   888.000000  
k_cover          220.000000   220.000000   220.000000  
rho_cover       2700.000000  2700.000000  2700.000000  
c_plate          888.000000   888.000000   888.000000  
k_plate          220.000000   220.000000   220.000000  
rho_plate       2700.000000  2700.000000  2700.000000  
channel_dist       0.040000     0.040000     0.040000  
cell_width         0.043000     0.043000     0.043000  
stoptime        2000.000000  2000.000000  2000.000000  
m_flow             0.003704     0.005556     0.016667  
area_cover         0.000512     0.000671     0.000859  
t_plate            0.028082     0.032397     0.035984  
area_plate         0.000133     0.000181     0.000256  
area_channel       0.000031     0.000050     0.000109  
perim_channel      0.029659     0.036867     0.049377

In [95]:
# save as individual json files
for d in df.to_dict(orient='index').values():
    simid = d["simid"]
    with open("../data/modelica_input_%s.json"%simid, "w", encoding="utf-8") as fp:
        json.dump(d, fp, indent=2)

In [71]:
df.to_csv("../data/200_samples_Vdot_const.csv", index=False)